# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.04878


extracting tarball to tmp_2302.04878...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.04880


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.04878/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tb_app' from 'tmp_2302.04878/tb_app.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'discussion' from 'tmp_2302.04878/discussion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/

...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.05341/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.05369...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.05370


extracting tarball to tmp_2302.05370...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04891-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04891) | **Demographics of Protoplanetary Disks: A Simulated Population of Edge-on  Systems**  |
|| Isabel Angelo, et al. -- incl., <mark>William J. Fischer</mark> |
|*Appeared on*| *2023-02-13*|
|*Comments*| *20 pages, 6 figures*|
|**Abstract**| The structure of protoplanetary disks plays an essential role in planet formation. Disks that are highly inclined, or ''edge-on'', are of particular interest since their geometry provides a unique opportunity to study the disk's vertical structure and radial extent. Candidate edge-on protoplanetary disks are typically identified via their unique spectral energy distribution and subsequently confirmed through high-resolution imaging. However, this selection process is likely biased toward the largest, most massive disks, and the resulting sample may not accurately represent the underlying disk population. To investigate this, we generated a grid of protoplanetary disk models using radiative transfer simulations and determined which sets of disk parameters produce edge-on systems that could be recovered by aforementioned detection techniques--i.e., identified by their spectral energy distribution and confirmed through follow-up imaging with HST. In doing so, we adopt a quantitative working definition of "edge-on disks" that is observation-driven and agnostic about the disk inclination or other properties. Folding in empirical disk demographics, we predict an occurrence rate of 6.2% for edge-on disks and quantify biases towards highly inclined, massive disks. We also find that edge-on disks are under-represented in samples of Spitzer-studied young stellar objects, particularly for disks with M $\lesssim$ 0.5 $M_\odot$. Overall, our analysis suggests that several dozen edge-on disks remain undiscovered in nearby star-forming regions, and provides a universal selection process to identify edge-on disks for consistent, population-level demographic studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05223-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05223) | **Protoplanetary Disk Science with the Orbiting Astronomical Satellite  Investigating Stellar Systems (OASIS) Observatory**  |
|| <mark>Kamber Schwarz</mark>, et al. |
|*Appeared on*| *2023-02-13*|
|*Comments*| *19 pages, 5 figures, published in Space Science Reviews*|
|**Abstract**| The Orbiting Astronomical Satellite for Investigating Stellar Systems (OASIS) is a NASA Astrophysics MIDEX-class mission concept, with the stated goal of following water from galaxies, through protostellar systems, to Earth's oceans. This paper details the protoplanetary disk science achievable with OASIS. OASIS's suite of heterodyne receivers allow for simultaneous, high spectral resolution observations of water emission lines spanning a large range of physical conditions within protoplanetary disks. These observations will allow us to map the spatial distribution of water vapor in disks across evolutionary stages and assess the importance of water, particularly the location of the midplane water snowline, to planet formation. OASIS will also detect the H2 isotopologue HD in 100+ disks, allowing for the most accurate determination of total protoplanetary disk gas mass to date. When combined with the contemporaneous water observations, the HD detection will also allow us to trace the evolution of water vapor across evolutionary stages. These observations will enable OASIS to characterize the time development of the water distribution and the role water plays in the process of planetary system formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04878-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04878) | **Designing an Optimal Kilonova Search using DECam for Gravitational Wave  Events**  |
|| C. R. Bom, et al. -- incl., <mark>K. Herner</mark> |
|*Appeared on*| *2023-02-13*|
|*Comments*| *23 pages, 10 figures, to be submitted to AAS Journals*|
|**Abstract**| We address the problem of optimally identifying all kilonovae detected via gravitational wave emission in the upcoming LIGO/Virgo/KAGRA Collaboration observing run, O4, which is expected to be sensitive to a factor of $\sim 7$ more Binary Neutron Stars alerts than previously. Electromagnetic follow-up of all but the brightest of these new events will require $>1$ meter telescopes, for which limited time is available. We present an optimized observing strategy for the Dark Energy Camera during O4. We base our study on simulations of gravitational wave events expected for O4 and wide-prior kilonova simulations. We derive the detectabilities of events for realistic observing conditions. We optimize our strategy for confirming a kilonova while minimizing telescope time. For a wide range of kilonova parameters, corresponding to a fainter kilonova compared to GW170817/AT2017gfo we find that, with this optimal strategy, the discovery probability for electromagnetic counterparts with the Dark Energy Camera is $\sim 80\%$ at the nominal binary neutron star gravitational wave detection limit for the next LVK observing run (190 Mpc), which corresponds to a $\sim 30\%$ improvement compared to the strategy adopted during the previous observing run. For more distant events ($\sim 330$ Mpc), we reach a $\sim 60\%$ probability of detection, a factor of $\sim 2$ increase. For a brighter kilonova model dominated by the blue component that reproduces the observations of GW170817/AT2017gfo, we find that we can reach $\sim 90\%$ probability of detection out to 330 Mpc, representing an increase of $\sim 20 \%$, while also reducing the total telescope time required to follow-up events by $\sim 20\%$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05261-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05261) | **Estimating AGN Black Hole Masses via Continuum Reverberation Mapping in  the Era of LSST**  |
|| Shu Wang, <mark>Hengxiao Guo</mark>, Jong-Hak Woo |
|*Appeared on*| *2023-02-13*|
|*Comments*| *10 pages, 2 figures, submitted to ApJL*|
|**Abstract**| Spectroscopic reverberation mapping (RM) is a direct approach widely used to estimate the mass of black holes (BHs) in active galactic nuclei (AGNs). However, it is very time-consuming and difficult to apply to a large AGN sample. The empirical relation between the broad-line region (BLR) size and luminosity (H$\beta$ $R_{\rm BLR}\unicode{x2013}L_{\rm}$) provides a practical alternative yet subject to large scatter and systematic bias. AGN continuum RM (CRM) recently discovered a similar relation between the continuum emitting region (CER) size and luminosity ($R_{\rm CER}\unicode{x2013}L$). Here we present a new method to estimate BH masses via optical CRM assuming continuum lags dominated by the diffuse continuum emission. This method will significantly facilitate the estimation of RM BH mass thanks to the short continuum lags and the easily accessible high-cadence and large-area photometric data. Using a sample of 21 AGNs with both CER and BLR size, we find a tight $R_{\rm BLR}\unicode{x2013}R_{\rm CER}$ relation (scatter $\sim0.28$ dex), with $R_{\rm BLR}$ on average 8.1 times of $R_{\rm CER}$ at 5100\r{A}. This tight relation enables the BH mass estimation based on the CRM combined with the velocity information. Applying the relation to rest objects in our CRM sample, we demonstrate that the predicted $R_{\rm BLR,CRM}$ follow the existing H$\beta$ $R_{\rm BLR}\unicode{x2013}L_{\rm}$ relation well and the estimated CRM BH masses are consistent with the RM/single-epoch BH masses using H$\beta$. Our proposed method will be a promising BH mass estimator in the era of Legacy Survey of Space and Time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05341-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05341) | **Further evidence that galaxy age drives observed type Ia supernova  luminosity differences**  |
|| P. Wiseman, M. Sullivan, <mark>M. Smith</mark>, B. Popovic |
|*Appeared on*| *2023-02-13*|
|*Comments*| *9 pages, 3 figures. Accepted for publication in MNRAS*|
|**Abstract**| Type Ia supernovae (SNe Ia) are explosions of white dwarf stars that facilitate exquisite measurements of cosmological expansion history, but improvements in accuracy and precision are hindered by observational biases. Of particular concern is the apparent difference in the corrected brightnesses of SNe Ia in different host galaxy environments. SNe Ia in more massive, passive, older environments appear brighter after having been standardized by their light-curve properties. The luminosity difference commonly takes the form of a step function. Recent works imply that environmental characteristics that trace the age of the stellar population in the vicinity of SNe show the largest steps. Here we use simulations of SN Ia populations to test the impact of using different tracers and investigate promising new models of the step. We test models with a total-to-selective dust extinction ratio $R_V$ that changes between young and old SN Ia host galaxies, as well as an intrinsic luminosity difference between SNe from young and old progenitors. The data are well replicated by a model driven by a galaxy-age varying $R_V$ and no intrinsic SN luminosity difference, and we find that specific star-formation rate measured locally to the SN is a relatively pure tracer of this galaxy age difference. We cannot rule out an intrinsic difference causing part of the observed step and show that if luminosity differences are caused by multiple drivers then no single environmental measurement is able to accurately trace them. We encourage the use of multiple tracers in luminosity corrections to negate this issue. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05369-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05369) | **Lyman-α polarization from cosmological ionization fronts: I.  Radiative transfer simulations**  |
|| Yuanyuan Yang, et al. -- incl., <mark>Heyang Long</mark> |
|*Appeared on*| *2023-02-13*|
|*Comments*| *29 pages, 13 figures, to be submitted to JCAP*|
|**Abstract**| In this paper, we present the formalism of simulating Lyman-$\alpha$ emission and polarization around reionization ($z$ = 8) from a plane-parallel ionization front. We accomplish this by using a Monte Carlo method to simulate the production of a Lyman-$\alpha$ photon, its propagation through an ionization front, and the eventual escape of this photon. This paper focuses on the relation of the input parameters of ionization front speed $U$, blackbody temperature $T_{\rm bb}$, and neutral hydrogen density $n_{\rm HI}$, on intensity $I$ and polarized intensity $P$ as seen by a distant observer. The resulting values of intensity range from $3.18\times 10^{-14}$ erg/cm$^{2}$/s/sr to $1.96 \times 10^{-9}$ erg/cm$^{2}$/s/sr , and the polarized intensity ranges from $5.73\times 10^{-17}$ erg/cm$^{2}$/s/sr to $5.31 \times 10^{-12}$ erg/cm$^{2}$/s/sr. We found that higher $T_{\rm bb}$, higher $U$, and higher $n_{\rm HI}$ contribute to higher intensity, as well as polarized intensity, though the strongest dependence was on the hydrogen density. The dependence of viewing angle of the front is also explored. We present tests to support the validity model, which makes the model suitable for further use in a following paper where we will calculate the intensity and polarized intensity power spectrum on a full reionization simulation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05370-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05370) | **Lyman-$α$ polarization from cosmological ionization fronts: II.  Implications for intensity mapping**  |
|| Emily Koivu, <mark>Heyang Long</mark>, Yuanyuan Yang, Christopher M. Hirata |
|*Appeared on*| *2023-02-13*|
|*Comments*| *18 pages, 9 figures, to be submitted to JCAP*|
|**Abstract**| This is the second paper in a series whose aim is to predict the power spectrum of intensity and polarized intensity from cosmic reionization fronts. After building the analytic models for intensity and polarized intensity calculations in paper I, here we apply these models to simulations of reionization. We construct a geometric model for identifying front boundaries, calculate the intensity and polarized intensity for each front, and compute a power spectrum of these results. This method was applied to different simulation sizes and resolutions, so we ensure that our results are convergent. We find that the power spectrum of fluctuations at $z=8$ in a bin of width $\Delta z=0.5$ ($\lambda/\Delta\lambda=18$) is $\Delta_\ell \equiv [\ell(\ell+1)C_\ell/2\pi]^{1/2}$ is $3.2\times 10^{-11}$ erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ for the intensity $I$, $7.6\times10^{-13}$ erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ for the $E$-mode polarization, and $5.8\times10^{-13}$ erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ for the $B$-mode polarization at $\ell=1.5\times10^4$. After computing the power spectrum, we compare results to detectable scales and discuss implications for observing this signal based on a proposed experiment. We find that, while fundamental physics does not exclude this kind of mapping from being attainable, an experiment would need to be highly ambitious and require significant advances to make mapping Lyman-$\alpha$ polarization from cosmic reionization fronts a feasible goal. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05026-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05026) | **Jet power, intrinsic $γ$-ray luminosity and accretion in jetted AGN**  |
|| Chen Yonhyun, et al. -- incl., <mark>Guo Xiaotong</mark> |
|*Appeared on*| *2023-02-13*|
|*Comments*| *10pages, 7fiures, accept for publication in MNRAS*|
|**Abstract**| The correlation between the kinetic jet power $P_{\rm jet}$, intrinsic $\gamma$-ray luminosity ($L^{\rm int}$) and accretion ($L_{\rm disk}$) may reveal the underlying jet physics in various black hole systems. We study the relation between kinetic jet power, intrinsic $\gamma$-ray luminosity, and accretion by using a large sample of jetted AGN, including flat-spectrum radio quasars (FSRQs), BL Lacertae objects (BL Lacs), gamma-ray Narrow-line Seyfert 1 galaxies ($\gamma$NLS1s) and radio galaxies. Our main results are as follows: (1) The slope indices of the relation between $P_{\rm jet}$ and $L^{\rm int}$ are $0.85\pm0.01$ for the whole sample, $0.70\pm0.02$ for the FSRQs, $0.83\pm0.03$ for the BL Lacs, $0.68\pm0.11$ for the $\gamma$NLS1s, and $0.93\pm0.09$ for the radio galaxies, respectively. The jets in $\gamma$NLS1s and radio galaxies almost follow the same $P_{\rm jet}$-$L^{\rm int}$ correlation that was obtained for Fermi blazars. (2) The slope indices of the relation between $L^{\rm int}$ and $L_{\rm disk}$ are $1.05\pm0.02$ for the whole sample, $0.94\pm0.05$ for the FSRQs, $1.14\pm0.05$ for the BL Lacs, and $0.92\pm0.18$ for the $\gamma$NLS1s, respectively. The $\gamma$NLS1s and radio galaxies almost also follow the $L^{\rm int}$-$L_{\rm disk}$ correlation derived for Fermi blazars. (3) The jet power is larger than the luminosity of accretion disks for almost all jetted AGN. Jet power depends on both the Eddington ratio and black hole mass. We obtain $\log P_{\rm jet}\sim(1.00\pm0.02)\log L_{\rm disk}$ for the whole sample, which is consistent with the theoretically predicted coefficient. These results may imply that the jets of jetted AGN are powered by the Blandford-Znajek mechanism. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

199  publications in the last 7 days.
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers